# Geographic Profiling with Open Street Map

In [29]:
import pandas
import geopandas as gpd
import osmnx as ox
import networkx as nx
from haversine import haversine
from shapely.geometry import Point, Polygon, LineString
from descartes import PolygonPatch
import matplotlib.pyplot as plt
import pdb

##### Choosing city and transportation mode

In [5]:
transport_mode = 'walk'
the_place = 'New-Orleans, LA'

##### Specify the path and read victims list

In [6]:
path2csv = 'victims.csv'
vics = pandas.read_csv(path2csv)

##### Filter victims

In [7]:
selection = (vics['Confirmed Axeman'] == 'Yes') | (vics['Confirmed Axeman'] == 'Maybe')
vicselect = vics.loc[selection,['Latitude','Longitude']]

##### The list of selected victims is

In [8]:
vics[selection]

,Victim,Time,Victims #,Attacked / Murdered / Break-in,Location,Latitude,Longitude,Confirmed Axeman
0,August Crutti,13/08/1910,1,Attacked,Royal St. and Lesseps St.,29.960853,-90.033930,Yes
1,Joseph Rissetto,20/09/1910,2,Attacked,Tonti St. and A.P. Tureaud Avenue,29.980195,-90.067196,Yes
2,Joseph Davi,26/06/1911,2,Murdered,Arts and Galvez St.,29.979005,-90.053034,Yes
7,Joseph Maggio,24/05/1918,2,Murdered,Upperline St. and Magnolia St.,29.937011,-90.107449,Yes
8,Louis Besumer,26/06/1918,2,Murdered,Dorgenois St. and Laharpe St.,29.977528,-90.076837,Maybe
9,Mary Schneider,05/08/1918,1,Attacked,1320 Elmira St.,29.941846,-90.047888,Yes
10,Joseph Romano,10/08/1918,3,Murdered,2336 Gravier St.,29.958806,-90.087881,Yes
14,Charles Cortimiglia,10/03/1919,3,Murdered,Second St. Gretna,29.916872,-90.062618,Yes
17,Mike Pepitone,27/10/1919,1,Murdered,S Scott St. and Ulloa St.,29.968005,-90.105312,Yes


In [23]:
lat = vicselect['Latitude']
lon = vicselect['Longitude']
N = len(lat) # Total number of victims

## Bounding box
The formula to get the boundaries of the bounding box are given in *Geographic Profiling* (2000) by Dr. Kim Rossmo:

$y_\text{high} = y_\text{max} + \left(C-1\right) \frac{y_\text{max}-y_\text{min}}{2}$

$y_\text{low} = y_\text{min} - \left(C-1\right) \frac{y_\text{max}-y_\text{min}}{2}$

$x_\text{high} = x_\text{max} + \left(C-1\right) \frac{x_\text{max}-x_\text{min}}{2}$

$x_\text{low} = x_\text{min} - \left(C-1\right) \frac{x_\text{max}-x_\text{min}}{2}$

Where:

$y_\text{high}$ is the $y$ value of the northernmost boundary 

$y_\text{low}$ is the $y$ value of the southermost boundary

$y_\text{max}$ is the maximum $y$ value for any crime site

$y_\text{min}$ is the minimum $y$ value for any crime site

$x_\text{high}$ is the $y$ value of the easternmost boundary

$x_\text{low}$ is the $y$ value of the westernmost boundary

$x_\text{max}$ is the maximum $x$ value for any crime site

$x_\text{min}$ is the minimum $x$ value for any crime site

and $C$ is the total number of crime sites

In [11]:
def get_bounding_box(vicselect):
    lat = vicselect['Latitude']
    lon = vicselect['Longitude']
    
    C = len(lat)
    
    ymax, ymin = max(lat), min(lat)
    xmax, xmin = max(lon), min(lon)
    
    yhigh = ymax + (C-1)*(ymax-ymin)/2.
    ylow  = ymin - (C-1)*(ymax-ymin)/2.
    xhigh = xmax + (C-1)*(xmax-xmin)/2.
    xlow  = xmin - (C-1)*(xmax-xmin)/2.
    
    return yhigh, ylow, xhigh, xlow

In [12]:
yhigh, ylow, xhigh, xlow = get_bounding_box(vicselect)

print(ylow, yhigh, xlow, xhigh)

29.663582700000013 30.233484299999986 -90.40152360000003 -89.73985529999996


#### Get the graph of the location

In [10]:
graph = ox.graph_from_place(the_place, network_type=transport_mode, simplify=False)
#ox.core.graph_from_bbox(north, south, east, west, network_type='all_private', simplify=True, retain_all=False, truncate_by_edge=False, name='unnamed', timeout=180, memory=None, max_query_area_size=2500000000, clean_periphery=True, infrastructure='way["highway"]', custom_filter=None)

nodes, edges = ox.graph_to_gdfs(graph, nodes=True, edges=True)

## Using Rossmo's formula

##### Define buffer/distance decay function:

$$f(d) = \begin{cases}
            \frac{k}{d^f} & \text{ if } d \gt B \\
            \frac{k B^{g-f}}{(2B - d)^g} & \text{ if } d \le B
        \end{cases} $$
        
Where $d$ is the distance from the anchor point, $k$ is constant, $B$ is the buffer zone radius and $g$ and $h$ are values describing the criminal's attributes e.g. the mode of travel they use.

In practice Rossmo in his thesis chooses $B$ to be one half the mean of the nearest neighbor distance between crimes (using a Manhattan metric). Also $f = 1.2$ was chosen from a gravity model formulation developed to describe interprovincial migration patterns of criminal fugtives (Rossmo, 1987, p. 136). The same exponent was used for distances both within and outside the buffer sone radius ($g = f$).

$k$ is only used to prevent "program variable overflow" (variable values must remain within the precision range of the specified integer and floating-point numeric variable types used). Basically just try different values and see what happens...

The distance-decay function could be improved by describing the buffer zone as an annulus instead of a radius. This could be achieved by using a third or fourth-order polynomial function.

In [13]:
def buffer_distance_decay(d,k,B,g,f):
    if d > B:
        return k/(d**f)
    else:
        return k*(B**(g-f))/((2*B-d)**g)


#### Score of a cell

For a cell (or node) $\mathbf{p}$, the score $S$ attributed is given by:

$$ S(\mathbf{p}) = \sum_{i=1}^{N} f\left(d\left(\mathbf{p},\mathbf{c_i}\right)\right) $$

Where $N$ is the total number of crimes; $d\left(\mathbf{p},\mathbf{c_i}\right)$ is the distance from cell $\mathbf{p}$ to crime $\mathbf{c_i}$. Rossmo uses a Manhattanian distance for $d$. Here, we use the actual walking distance depending on the city network.

#### Defining distances

In [31]:
def get_path_length(graph, orig_node, target_node):
    length = nx.shortest_path_length(G=graph, source=orig_node, target=target_node, weight='length')
    return length

def get_manhattan_haversine(point1,point2):
    # point = (lat,lon)
    # haversine((lat[0],lon[0]),(lat[1],lon[1]),unit="m")
    # returns the manhattan distance using haversine formula
    return abs(haversine((0,point1[1]),(0,point2[1]),unit="m")) + abs(haversine((point1[0],0),(point2[0],0),unit="m"))

In [36]:
print(get_manhattan_haversine((lat[0],lon[0]),(lat[1],lon[1])),haversine((lat[0],lon[0]),(lat[1],lon[1]),unit="m"))

5849.65070535537 3859.19015306046


## Using a Dirichlet process mixture model